# Model creation and data split-up

This notebook will:
* Split data ito test and train data
* Train the model
* Save the model

Import packages

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pds

Import data

In [2]:
df = pds.read_pickle('ANN_dataset.pkl')
df.head()

,timestamp,lon,lat,direction,speed,journey,seg,time_left,segment_time,tsjs
0,2018-02-16 04:48:40+01:00,58.414238,15.571015,147.300003,0.0,1,1,71.0,71.0,0.0
1,2018-02-16 04:48:41+01:00,58.414246,15.571012,147.300003,0.0,1,1,70.0,71.0,0.0
2,2018-02-16 04:48:42+01:00,58.414249,15.571008,147.300003,0.0,1,1,69.0,71.0,0.0
3,2018-02-16 04:48:43+01:00,58.414257,15.571004,147.300003,0.0,1,1,68.0,71.0,0.0
4,2018-02-16 04:48:44+01:00,58.414257,15.571006,147.300003,0.0,1,1,67.0,71.0,0.0


Note that `segment_time` is the label in this model 

In [3]:
label_string = 'segment_time'

One hot encode segments, normalize columns and create final dataset

In [4]:
# Make segments categorical
dataset = pds.get_dummies(df['seg'])

# Combine segments, timestamps and time from journey start

# Cyclical timestamps 
dataset['hr_sin'] = np.sin(df.timestamp.dt.hour*(2.*np.pi/24))
dataset['hr_cos'] = np.cos(df.timestamp.dt.hour*(2.*np.pi/24))

dataset['tsjs'] = (df['tsjs']-df['tsjs'].min())/(df['tsjs'].max()-df['tsjs'].min())

# Include journey number to select entire journeys
dataset['journey'] = df['journey']
dataset.head()

,1,2,3,4,5,6,7,8,9,10,11,hr_sin,hr_cos,tsjs,journey
0,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.0,1
1,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.0,1
2,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.0,1
3,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.0,1
4,1,0,0,0,0,0,0,0,0,0,0,0.866025,0.5,0.0,1


Select ~20% test journeys

In [5]:
divider = 0.2
num_journeys = dataset.journey.unique()[-1]
num_test_segments = np.int_(np.round(num_journeys*divider))
msk = np.random.randint(1, num_journeys, num_test_segments)
mask = dataset['journey'].isin(msk)

Divide training and test data, where data is a sequence from a journey

In [6]:
train_input = dataset[~mask]
test_input = dataset[mask]

train_labels = df[label_string][~mask]
test_labels = df[label_string][mask]

In [7]:
# Journey numbers were needed up until this point
train_data = train_input.drop(columns=['journey'])
test_data = test_input.drop(columns=['journey'])

Build model

In [8]:
model = keras.Sequential([
	keras.layers.Dense(2*len(train_data.columns), activation=tf.nn.tanh, input_shape=(train_data.shape[1],)),
    keras.layers.Dense(len(train_data.columns), activation=tf.nn.relu),
	keras.layers.Dense(1, activation=tf.nn.relu)
	])

optimizer = keras.optimizers.Adadelta()
model.compile(loss='mae', optimizer = optimizer, metrics=['mae'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 28)                420       
_________________________________________________________________
dense_1 (Dense)              (None, 14)                406       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 15        
Total params: 841
Trainable params: 841
Non-trainable params: 0
_________________________________________________________________
None


Fit model, 10% of all training data is used as validation data

In [9]:
EPOCHS=5
model.fit(train_data.values, train_labels.values, epochs = EPOCHS, validation_split=0.1, initial_epoch=0)

Train on 2433164 samples, validate on 270352 samples
Epoch 1/5
2433164/2433164 [==============================] - 57s 23us/step - loss: 17.5803 - mean_absolute_error: 17.5803 - val_loss: 17.8480 - val_mean_absolute_error: 17.8480
Epoch 2/5
2433164/2433164 [==============================] - 61s 25us/step - loss: 17.3768 - mean_absolute_error: 17.3768 - val_loss: 17.7433 - val_mean_absolute_error: 17.7433
Epoch 3/5
2433164/2433164 [==============================] - 62s 25us/step - loss: 17.3274 - mean_absolute_error: 17.3274 - val_loss: 17.8794 - val_mean_absolute_error: 17.8794
Epoch 4/5
2433164/2433164 [==============================] - 61s 25us/step - loss: 17.3028 - mean_absolute_error: 17.3028 - val_loss: 17.7824 - val_mean_absolute_error: 17.7824
Epoch 5/5
2433164/2433164 [==============================] - 60s 25us/step - loss: 17.2854 - mean_absolute_error: 17.2854 - val_loss: 17.9249 - val_mean_absolute_error: 17.9249


Evaluate model on all test data

In [10]:
model.evaluate(test_data.values, test_labels.values)

608484/608484 [==============================] - 6s 10us/step


[17.803824904618228, 17.803824904618228]

Prepare data for the evaluation step

In [11]:
# Same name for all files
model_name = 'ANN_M1'

# Serialize model to YAML
model_yaml = model.to_yaml()
with open(model_name + '.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)

# Serialize weights to HDF5
model.save_weights(model_name + '.h5')

In [12]:
output = pds.concat([test_input, test_labels], axis=1)
output.head()

,1,2,3,4,5,6,7,8,9,10,11,hr_sin,hr_cos,tsjs,journey,segment_time
20089,1,0,0,0,0,0,0,0,0,0,0,0.5,-0.866025,0.0,23,225.0
20090,1,0,0,0,0,0,0,0,0,0,0,0.5,-0.866025,0.0,23,225.0
20091,1,0,0,0,0,0,0,0,0,0,0,0.5,-0.866025,0.0,23,225.0
20092,1,0,0,0,0,0,0,0,0,0,0,0.5,-0.866025,0.0,23,225.0
20093,1,0,0,0,0,0,0,0,0,0,0,0.5,-0.866025,0.0,23,225.0


In [13]:
output.to_pickle(model_name + '.pkl')

Done! Get ready for evaluation